In [1]:
%matplotlib widget

import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
import pandas as pd
from amftrack.util import get_dates_datetime, get_dirname, get_data_info, update_plate_info, \
get_current_folders, get_folders_by_plate_id
import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.paths.directory import *


In [2]:
directory = directory_project
update_plate_info(directory)

In [2]:
directory = directory_scratch+'agg/'
update_plate_info(directory)

In [4]:
len(get_current_folders(directory))
all_folders = get_current_folders(directory)

In [5]:
all_folders['Plate']=all_folders['Plate'].fillna(758)

In [6]:
folders = all_folders.loc[(all_folders['/Analysis/nx_graph_pruned_width.p']==True)]
folders

,folder,Plate,PrincePos,root,strain,medium,split,Temp,CrossDate,/Img/TileConfiguration.txt.registered,...,/Analysis/skeleton_masked_compressed.mat,/Analysis/skeleton_pruned_compressed.mat,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,Pbait
1758,20210912_0413_Plate22,756,22,Carrot,Agg,001P100N,Y,NaN,20210907,True,...,True,True,True,False,True,True,True,False,"12.09.2021, 04:13:",-
1760,20210915_0319_Plate22,756,22,Carrot,Agg,001P100N,Y,NaN,20210907,True,...,True,True,True,False,True,True,True,False,"15.09.2021, 03:19:",-
1762,20210911_1813_Plate22,756,22,Carrot,Agg,001P100N,Y,NaN,20210907,True,...,True,True,True,False,True,True,True,False,"11.09.2021, 18:13:",-
1774,20210910_1013_Plate22,756,22,Carrot,Agg,001P100N,Y,NaN,20210907,True,...,True,True,True,False,True,True,True,False,"10.09.2021, 10:13:",-
1788,20210910_0213_Plate22,756,22,Carrot,Agg,001P100N,Y,NaN,20210907,True,...,True,True,True,False,True,True,True,False,"10.09.2021, 02:13:",-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2846,20211019_2325_Plate17,758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,...,True,True,True,False,True,True,True,False,"19.10.2021, 23:25:",NaN
2848,20211014_0348_Plate17,758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,...,True,True,True,False,True,True,True,False,"14.10.2021, 03:48:",NaN
2849,20211009_1737_Plate17,758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,...,True,True,True,False,True,True,True,False,"09.10.2021, 17:37:",NaN
2850,20211011_1346_Plate17,758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,...,True,True,True,False,True,True,True,False,"11.10.2021, 13:46:",NaN


In [5]:
folders = all_folders.loc[(all_folders['/Analysis/nx_graph_pruned_width.p']==False)&(all_folders['/Analysis/nx_graph_pruned.p']==True)]
folders

,folder,Plate,PrincePos,root,strain,medium,split,Temp,CrossDate,/Img/TileConfiguration.txt.registered,...,/Analysis/skeleton_masked_compressed.mat,/Analysis/skeleton_pruned_compressed.mat,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,Pbait
2111,20210911_1213_Plate22,756,22,Carrot,Agg,001P100N,Y,NaN,20210907,True,...,True,True,True,False,True,True,False,False,"11.09.2021, 12:13:",-


In [4]:
plates = [799,22,80,10,76,94,88,26,808,792,91,785,787,800,152,790]
folders = all_folders.loc[(all_folders['strain']=='A5')&(all_folders['/Analysis/nx_graph_pruned_width.p'])&(all_folders['Plate'].isin(plates)==0)]
folders

,folder,Plate,PrincePos,root,strain,medium,split,Temp,CrossDate,/Img/TileConfiguration.txt.registered,...,/Analysis/skeleton_masked_compressed.mat,/Analysis/skeleton_pruned_compressed.mat,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,Pbait
8,20201205_1320_Plate38,59,38,Carrot,A5,001P100N,Y,NaN,20201126,True,...,True,True,True,False,True,True,True,False,"05.12.2020, 13:20:",-
12,20201206_1327_Plate40,69,40,Carrot,A5,001P100N,Y,25,20201119,True,...,True,True,True,False,True,True,True,False,"06.12.2020, 13:27:",NaN
30,20201207_0526_Plate40,69,40,Carrot,A5,001P100N,Y,25,20201119,True,...,True,True,True,False,True,True,True,False,"07.12.2020, 05:26:",NaN
48,20201229_0349_Plate40,102,40,Carrot,A5,001P100N,Y,25,20201226,True,...,True,True,True,False,True,True,True,True,"29.12.2020, 03:49:",NaN
49,20210112_1143_Plate40,102,40,Carrot,A5,001P100N,Y,25,20201226,True,...,True,True,True,False,True,True,True,False,"12.01.2021, 11:43:",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1740,20201124_0404_Plate40,69,40,Carrot,A5,001P100N,Y,25,20201119,True,...,True,True,True,False,True,True,True,False,"24.11.2020, 04:04:",NaN
1742,20201120_1621_Plate40,69,40,Carrot,A5,001P100N,Y,25,20201119,True,...,True,True,True,False,True,True,True,False,"20.11.2020, 16:21:",NaN
1748,20201120_0822_Plate40,69,40,Carrot,A5,001P100N,Y,25,20201119,True,...,True,True,True,False,True,True,True,False,"20.11.2020, 08:22:",NaN
1750,20201123_1222_Plate40,69,40,Carrot,A5,001P100N,Y,25,20201119,True,...,True,True,True,False,True,True,True,False,"23.11.2020, 12:22:",NaN


In [10]:
folders = all_folders.loc[(all_folders['strain']=='A5')&(all_folders['/Analysis/nx_graph_pruned_labeled.p'])&(all_folders['Plate']==102)]
folders

,folder,Plate,PrincePos,root,strain,medium,split,Temp,CrossDate,/Img/TileConfiguration.txt.registered,...,/Analysis/skeleton_masked_compressed.mat,/Analysis/skeleton_pruned_compressed.mat,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,Pbait
48,20201229_0349_Plate40,102,40,Carrot,A5,001P100N,Y,25,20201226,True,...,True,True,True,False,True,True,True,True,"29.12.2020, 03:49:",NaN
56,20201230_2333_Plate40,102,40,Carrot,A5,001P100N,Y,25,20201226,True,...,True,True,True,False,True,True,True,True,"30.12.2020, 23:33:",NaN
71,20210103_2332_Plate40,102,40,Carrot,A5,001P100N,Y,25,20201226,True,...,True,True,True,False,True,True,True,True,"03.01.2021, 23:32:",NaN
95,20201229_2336_Plate40,102,40,Carrot,A5,001P100N,Y,25,20201226,True,...,True,True,True,False,True,True,True,True,"29.12.2020, 23:36:",NaN
107,20201227_0803_Plate40,102,40,Carrot,A5,001P100N,Y,25,20201226,True,...,True,True,True,False,True,True,True,True,"27.12.2020, 08:03:",NaN
123,20201229_1137_Plate40,102,40,Carrot,A5,001P100N,Y,25,20201226,True,...,True,True,True,False,True,True,True,True,"29.12.2020, 11:37:",NaN
167,20201226_2004_Plate40,102,40,Carrot,A5,001P100N,Y,25,20201226,True,...,True,True,True,False,True,True,True,True,"26.12.2020, 20:04:",NaN
176,20210102_1133_Plate40,102,40,Carrot,A5,001P100N,Y,25,20201226,True,...,True,True,True,False,True,True,True,True,"02.01.2021, 11:33:",NaN
198,20210104_1952_Plate40,102,40,Carrot,A5,001P100N,Y,25,20201226,True,...,True,True,True,False,True,True,True,True,"04.01.2021, 19:52:",NaN
200,20201228_0750_Plate40,102,40,Carrot,A5,001P100N,Y,25,20201226,True,...,True,True,True,False,True,True,True,True,"28.12.2020, 07:50:",NaN


In [29]:
plate = 22
plates = [22,10,76,26,152,102]

# folders = all_folders.loc[all_folders['Plate']==plate]
folders = all_folders.loc[all_folders['Plate'].isin(plates)]
print(len(folders))
# folders = all_folders.loc[all_folders['Plate']==plate*all_folders['/Analysis/skeleton_realigned_compressed.mat']]
folders = all_folders.loc[(all_folders['/Analysis/skeleton_realigned_compressed.mat'])&(all_folders['Plate'].isin(plates))]

print(len(folders))

351
336


In [10]:
folders = all_folders.loc[(all_folders['strain']=='A5')&(all_folders['/Analysis/nx_graph_pruned_width.p'])]
len(folders)


1748

In [4]:
folders = all_folders.loc[(all_folders['Plate']==792)&(all_folders['/Analysis/skeleton_realigned_compressed.mat'])]
# folders = folders.loc[(folders['/Analysis/nx_graph_pruned_width.p']==False)]

In [5]:
folders = all_folders.loc[(all_folders['/Analysis/skeleton_realigned_compressed.mat'])]
# folders = folders.loc[(folders['/Analysis/nx_graph_pruned_width.p']==False)]

In [5]:
# folders = all_folders.loc[(all_folders['/Analysis/nx_graph_pruned.p'])]
folders

,folder,Plate,PrincePos,root,strain,medium,split,Temp,CrossDate,/Img/TileConfiguration.txt.registered,...,/Analysis/skeleton_masked_compressed.mat,/Analysis/skeleton_pruned_compressed.mat,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,Pbait
1763,20210918_1729_Plate21,792,21,Carrot,A5,001P100N,Y,NaN,20210915,True,...,True,True,True,False,True,True,True,True,"18.09.2021, 17:29:",-
1766,20210919_0129_Plate21,792,21,Carrot,A5,001P100N,Y,NaN,20210915,True,...,True,True,True,False,True,True,True,True,"19.09.2021, 01:29:",-
1770,20211001_0323_Plate21,792,21,Carrot,A5,001P100N,Y,NaN,20210915,True,...,True,True,True,False,True,True,True,False,"01.10.2021, 03:23:",-
1771,20210915_2318_Plate21,792,21,Carrot,A5,001P100N,Y,NaN,20210915,True,...,True,True,True,False,True,True,True,True,"15.09.2021, 23:18:",-
1776,20210922_1320_Plate21,792,21,Carrot,A5,001P100N,Y,NaN,20210915,True,...,True,True,True,False,True,True,True,True,"22.09.2021, 13:20:",-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2662,20210920_0329_Plate21,792,21,Carrot,A5,001P100N,Y,NaN,20210915,True,...,True,True,True,False,True,True,True,True,"20.09.2021, 03:29:",-
2665,20211011_1353_Plate21,792,21,Carrot,A5,001P100N,Y,NaN,20210915,True,...,True,True,True,False,True,True,True,False,"11.10.2021, 13:53:",-
2682,20210928_0308_Plate21,792,21,Carrot,A5,001P100N,Y,NaN,20210915,True,...,True,True,True,False,True,True,True,False,"28.09.2021, 03:08:",-
2686,20210917_1318_Plate21,792,21,Carrot,A5,001P100N,Y,NaN,20210915,True,...,True,True,True,False,True,True,True,True,"17.09.2021, 13:18:",-


In [51]:
corrupted_rotation

1047    20201209_0926_Plate40
Name: folder, dtype: object

In [53]:
corrupted_rotation = folders.loc[folders['/Analysis/transform_corrupt.mat']]['folder']

In [28]:
corrupt_folder = np.min(folders.loc[folders['/Analysis/transform_corrupt.mat']]['folder'])
corrupt_folder

'20210927_2112_Plate22'

In [54]:
folder_list = list(folders['folder'])
folder_list.sort()
indexes = [folder_list.index(corrupt_folder) for corrupt_folder in corrupted_rotation]

In [55]:
limit = 100
indexes.sort()
indexes = [index for index in indexes if index<limit]
indexes += [limit]
indexes

[100]

In [29]:

folder_list.index(corrupt_folder)

23

In [36]:
folders = all_folders.loc[all_folders['folder']=='20211009_0617_Plate38']
folders

,folder,Plate,PrincePos,root,strain,medium,split,Temp,CrossDate,/Img/TileConfiguration.txt.registered,...,/Analysis/skeleton_masked_compressed.mat,/Analysis/skeleton_pruned_compressed.mat,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,Pbait
1816,20211009_0617_Plate38,785,38,Carrot,A5,001P100N,Y,NaN,20210928,True,...,True,True,True,False,True,True,True,False,"09.10.2021, 06:17:",-


In [13]:
plates = set(folders['Plate'].values)
plate_without_node_id = [plate for plate in plates if not np.any(folders.loc[(folders['Plate']==plate)]['/Analysis/nx_graph_pruned_labeled.p'])]

In [14]:
plate_without_node_id

[]

In [8]:
folders = all_folders.loc[(all_folders['Plate'].isin(plate_without_node_id))&(all_folders['/Analysis/skeleton_realigned_compressed.mat'])]

In [11]:
dates = list(all_folders.loc[all_folders['Plate']==785]['folder'])
dates.sort()
dates

['20210929_1004_Plate38',
 '20210929_1204_Plate38',
 '20210929_1404_Plate38',
 '20210929_1604_Plate38',
 '20210929_1804_Plate38',
 '20210929_2004_Plate38',
 '20210929_2204_Plate38',
 '20210930_0004_Plate38',
 '20210930_0204_Plate38',
 '20210930_0404_Plate38',
 '20210930_0604_Plate38',
 '20210930_0804_Plate38',
 '20210930_1004_Plate38',
 '20210930_2029_Plate38',
 '20210930_2210_Plate38',
 '20211001_0210_Plate38',
 '20211001_0410_Plate38',
 '20211001_0610_Plate38',
 '20211001_0810_Plate38',
 '20211001_1010_Plate38',
 '20211001_1210_Plate38',
 '20211002_1543_Plate38',
 '20211002_1744_Plate38',
 '20211002_1944_Plate38',
 '20211002_2144_Plate38',
 '20211002_2344_Plate38',
 '20211003_0143_Plate38',
 '20211003_0344_Plate38',
 '20211003_0543_Plate38',
 '20211003_0744_Plate38',
 '20211003_0944_Plate38',
 '20211003_1144_Plate38',
 '20211003_1344_Plate38',
 '20211003_1544_Plate38',
 '20211003_1744_Plate38',
 '20211003_1944_Plate38',
 '20211003_2144_Plate38',
 '20211003_2344_Plate38',
 '20211004_0

***Create graphs***

In [9]:
num_parallel = 25
time = '2:00:00'
args=[directory]
run_parallel('extract_nx_graph.py',args, folders, num_parallel, time,'extract_nx',cpus = 32)

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 179319
Submitted batch job 179320
Submitted batch job 179321
Submitted batch job 179322
Submitted batch job 179323
Submitted batch job 179324
Submitted batch job 179325


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [69]:
plates = [22,10,76,26,152,102]

folders = all_folders.loc[(all_folders['/Analysis/nx_graph_pruned.p'])&(all_folders['Plate'].isin(plates))]
len(folders)

336

***Extract Width***

In [11]:
num_parallel = 6
time = '24:00:00'
skip = False
resolution = 50
args = [directory,skip,resolution]
run_parallel('extract_width.py',args,folders,num_parallel,time,'extract_width2',cpus=32)

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 183068
Submitted batch job 183069


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [41]:
plates = [22,10,76,26,152,102]

folders = all_folders.loc[(1-all_folders['/Analysis/nx_graph_pruned_width.p'])&(all_folders['Plate'].isin(plates))&(all_folders['/Analysis/nx_graph_pruned.p'])]
# folders.loc[folders['folder'] == '20210110_1523_Plate30']
# int(list(folders['folder'])[0].split('_')[-1][5:])
len(folders)

0

In [4]:
plates = [22,10,76,26,152,102]

folders = all_folders.loc[(all_folders['/Analysis/nx_graph_pruned_width.p'])&(all_folders['Plate'].isin(plates))]
# folders.loc[folders['folder'] == '20210110_1523_Plate30']
# int(list(folders['folder'])[0].split('_')[-1][5:])
len(folders)

336

***Identify Nodes***

In [7]:
num_parallel = 1
time = '48:00:00'
limit = 25
args=[directory,limit]
run_parallel_all_time('extract_nodes.py',args,folders,num_parallel,time,'node_id_102',cpus = 32)

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 192634
Submitted batch job 192635
Submitted batch job 192636


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [60]:
plates = [22,10,76,26,152,102]

folders = all_folders.loc[(all_folders['/Analysis/nx_graph_pruned_labeled.p'])&(all_folders['Plate'].isin(plates))]
# folders.loc[folders['folder'] == '20210110_1523_Plate30']
# int(list(folders['folder'])[0].split('_')[-1][5:])
len(folders)

239

***Hyphae extraction***

In [9]:
num_parallel = 1
time = '4:00:00'
limit = 25
version = 9
args = [directory,limit,version]
run_parallel_all_time('hyphae_extraction.py',args,folders,num_parallel,time,'hyphae',cpus = 32)

Submitted batch job 194042
Submitted batch job 194043
Submitted batch job 194044


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
